# Getting Processed data notebooks
Purpose: to access processed data and combine for tabular, notes, and images

This will be for early concatenation and early fusion models

Currently accessing locally --> convert to EC2 and S3 access ASAP!

In [1]:
import pandas as pd
import numpy as np
import json
from glob import glob
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain, MultiOutputClassifier
from xgboost import XGBClassifier

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

rand_state = 42

In [4]:
import sklearn 
print(sklearn.__version__) 

1.2.2


# Notes and Tabular Data

In [23]:
#notes data - currently pulling locally from google drive (need to mount to S3 and pull from there to use EC2 in the future for larger models!)
data_dir_processed = r"C:/Users/Carolyn/Documents/MIDS/210 Capstone/train_validation_test__datasets"

In [5]:
# Define the names of the JSON files
#train_df_filename = "/train_set__chexpert__4_findings__single_label__unbalanced.json"
#test_df_filename = "/test_set__chexpert__4_findings__single_label__unbalanced.json"
#valid_df_filename = "/validation_set__chexpert__4_findings__single_label__unbalanced.json"

#train_balanced_df_filename = "/train_set__chexpert__4_findings__single_label__balanced.json"

def load_and_combine_json_files(directory_path, search_pattern):
    # Use glob to find JSON files in the directory based on the search pattern
    pop_files = glob(directory_path + search_pattern)

    # Initialize an empty DataFrame to hold all the data
    combined_df = pd.DataFrame()

    # Loop through each JSON file
    for file in pop_files:
        # Determine the dataset type based on the file name
        if 'train' in file:
            dataset_type = 'train'
        elif 'test' in file:
            dataset_type = 'test'
        elif 'val' in file:
            dataset_type = 'validate'
        else:
            dataset_type = 'unknown'

        print('Loading data files...', file, dataset_type)
        # Load the JSON file into a DataFrame
        with open(file) as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        if dataset_type == 'train':
            train_df = df
        elif dataset_type == 'test':
            test_df = df
        elif dataset_type == 'validate':
            val_df = df
        else:
            print("Unknown dataset type!")

        # Add a new column to flag the dataset type
        df['dataset_type'] = dataset_type

        # Append the DataFrame to the combined DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    return train_df, test_df, val_df, combined_df

# Define parameters for the function to combine JSON files
directory_path = 'C:/Users/Carolyn/Documents/MIDS/210 Capstone/fusion_data/'
pop_files = '*.json'  # This pattern can be changed based on the files you're looking for

# Load and combine the JSON files
train_df, test_df, val_df, combined_df = load_and_combine_json_files(directory_path, pop_files)

print('Total Cols\n',train_df.columns)

Loading data files... C:/Users/Carolyn/Documents/MIDS/210 Capstone/fusion_data\test_set__chexpert__4_findings__single_label__unbalanced.json test
Loading data files... C:/Users/Carolyn/Documents/MIDS/210 Capstone/fusion_data\train_set__chexpert__4_findings__single_label__unbalanced.json train
Loading data files... C:/Users/Carolyn/Documents/MIDS/210 Capstone/fusion_data\validation_set__chexpert__4_findings__single_label__unbalanced.json validate
Total Cols
 Index(['patient_id', 'visit_id', 'study_id', 'temperature', 'heartrate',
       'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity',
       'positive_label_total', 'finding_names', 'radiology_note',
       'discharge_note', 'chief_complaint',
       'major_surgical_or_invasive_procedure', 'history_of_present_illness',
       'past_medical_history', 'family_history', 'atelectasis', 'cardiomegaly',
       'lung_opacity', 'pleural_effusion', 'dataset_type'],
      dtype='object')


In [6]:
train_df.head()

,patient_id,visit_id,study_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,positive_label_total,finding_names,radiology_note,discharge_note,chief_complaint,major_surgical_or_invasive_procedure,history_of_present_illness,past_medical_history,family_history,atelectasis,cardiomegaly,lung_opacity,pleural_effusion,dataset_type
0,10000935.0,26381316,51178377.0,97.6,117.0,18.0,95.0,128.0,74.0,10,3.0,1.0,lung_opacity,FINAL REPORT\...,\nName: ___ Unit No: ___...,"Weakness, nausea/vomiting",none,This is a ___ yo f with h/o recently diagnosed...,PMH: \n# high grade SBO ___ s/p exploratory la...,Mother - ___ cancer d. at ___ \nYoungest of _...,0.0,0.0,1.0,0.0,train
1,10000980.0,29654838,59988438.0,97.8,57.0,18.0,100.0,180.0,88.0,0,2.0,1.0,pleural_effusion,FINAL REPORT\...,\nName: ___ Unit No: ___\n \nAdmi...,Shortness of breath,None,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",Denies cardiac family history. Family hx of DM...,0.0,0.0,0.0,1.0,train
2,10001176.0,23334588,53186264.0,101.3,97.0,18.0,93.0,168.0,58.0,6,3.0,1.0,lung_opacity,FINAL REPORT\...,\nName: ___ Unit No: ___...,fever,none,"___ with history of morbid obesity, coronary a...",MYOCARDIAL INFARCT - INFEROPOSTERIOR \nHYPERC...,Non contributory,0.0,0.0,1.0,0.0,train
3,10001217.0,24597018,52067803.0,99.0,81.0,16.0,97.0,160.0,102.0,0,3.0,1.0,atelectasis,FINAL REPORT\...,\nName: ___ Unit No: ___\n \n...,"Left hand and face numbness, left hand weaknes...",Right parietal craniotomy for abscess incision...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis,"Mother with pancreatic cancer, brother-lung ca...",1.0,0.0,0.0,0.0,train
4,10001401.0,26840593,51065211.0,97.8,67.0,20.0,95.0,90.0,43.0,8,2.0,1.0,atelectasis,FINAL REPORT\...,\nName: ___ Unit No: ___\n \n...,"Abdominal pain, distention, nausea",Interventional radiology placement of abdomina...,"___ F with h/o muscle invasive bladder cancer,...","Hypertension, laparoscopic cholecystectomy, le...",Negative for bladder CA.,1.0,0.0,0.0,0.0,train


In [7]:
def return_column_values_sum_and_percentage(dataframe_input, column_input):
    total_sum = dataframe_input[column_input].sum()
    percentages = dataframe_input[column_input] / total_sum
    sums_percentages = pd.DataFrame({
        'sum': dataframe_input[column_input],
        'percentage': percentages
    })
    sums_percentages['cumsum_percentage'] = sums_percentages['percentage'].cumsum()
    sums_percentages['sum'] = sums_percentages['sum'].apply(lambda x: "{:,}".format(x))
    sums_percentages['percentage'] = sums_percentages['percentage'].mul(100).round(1).astype(str) + '%'
    sums_percentages['cumsum_percentage'] = sums_percentages['cumsum_percentage'].mul(100).round(1).astype(str) + '%'
    return sums_percentages

In [8]:
return_column_values_sum_and_percentage(train_df.groupby("finding_names").agg({"study_id": "count"}).sort_values("study_id", ascending=False), "study_id")

,sum,percentage,cumsum_percentage
finding_names,,,
no_finding,"5,530",71.3%,71.3%
lung_opacity,893,11.5%,82.8%
cardiomegaly,516,6.7%,89.4%
atelectasis,466,6.0%,95.4%
pleural_effusion,353,4.6%,100.0%


In [ ]:
#HAIM biobert embeddings test
def get_biobert_embeddings(text):
    # Inputs:
    #   text -> Input text (str)
    #
    # Outputs:
    #   embeddings -> Final Biobert embeddings with vector dimensionality = (1,768)
    #   hidden_embeddings -> Last hidden layer in Biobert model with vector dimensionality = (token_size,768)
  
    # %% EXAMPLE OF USE
    # embeddings, hidden_embeddings = get_biobert_embeddings(text)
  
    tokens_pt = biobert_tokenizer(text, return_tensors="pt")
    outputs = biobert_model(**tokens_pt)
    last_hidden_state = outputs.last_hidden_state
    pooler_output = outputs.pooler_output
    hidden_embeddings = last_hidden_state.detach().numpy()
    embeddings = pooler_output.detach().numpy()

    return embeddings, hidden_embeddings


# Images

In [ ]:
#